In [11]:
#import dependencies 
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [12]:
data = pd.read_csv('COVID-19/data/italy-province.csv')
data.head()

,data,stato,codice_regione,denominazione_regione,codice_provincia,denominazione_provincia,sigla_provincia,lat,long,totale_casi
0,2020-02-24 18:00:00,ITA,13,Abruzzo,69,Chieti,CH,42.351032,14.167546,0.0
1,2020-02-24 18:00:00,ITA,13,Abruzzo,66,L'Aquila,AQ,42.351222,13.398438,0.0
2,2020-02-24 18:00:00,ITA,13,Abruzzo,68,Pescara,PE,42.464584,14.213648,0.0
3,2020-02-24 18:00:00,ITA,13,Abruzzo,67,Teramo,TE,42.658918,13.704400,0.0
4,2020-02-24 18:00:00,ITA,13,Abruzzo,979,In fase di definizione/aggiornamento,NaN,0.000000,0.000000,0.0


In [13]:
data_clean = data.drop(['stato', 'codice_regione', 'codice_provincia', 'lat', 'long', 'sigla_provincia'], axis=1)
data_clean

,data,denominazione_regione,denominazione_provincia,totale_casi
0,2020-02-24 18:00:00,Abruzzo,Chieti,0.0
1,2020-02-24 18:00:00,Abruzzo,L'Aquila,0.0
2,2020-02-24 18:00:00,Abruzzo,Pescara,0.0
3,2020-02-24 18:00:00,Abruzzo,Teramo,0.0
4,2020-02-24 18:00:00,Abruzzo,In fase di definizione/aggiornamento,0.0
...,...,...,...,...
2427,2020-03-13 17:00:00,Veneto,Treviso,327.0
2428,2020-03-13 17:00:00,Veneto,Venezia,248.0
2429,2020-03-13 17:00:00,Veneto,Verona,210.0
2430,2020-03-13 17:00:00,Veneto,Vicenza,141.0


In [14]:
#Translate column headers from Italian to English 
data_english = data_clean.rename(columns={'data': 'DATE', 'denominazione_regione': 'REGION NAME', 'denominazione_provincia': 'PROVINCE NAME', 'totale_casi': 'TOTAL CASES'})
data_english

,DATE,REGION NAME,PROVINCE NAME,TOTAL CASES
0,2020-02-24 18:00:00,Abruzzo,Chieti,0.0
1,2020-02-24 18:00:00,Abruzzo,L'Aquila,0.0
2,2020-02-24 18:00:00,Abruzzo,Pescara,0.0
3,2020-02-24 18:00:00,Abruzzo,Teramo,0.0
4,2020-02-24 18:00:00,Abruzzo,In fase di definizione/aggiornamento,0.0
...,...,...,...,...
2427,2020-03-13 17:00:00,Veneto,Treviso,327.0
2428,2020-03-13 17:00:00,Veneto,Venezia,248.0
2429,2020-03-13 17:00:00,Veneto,Verona,210.0
2430,2020-03-13 17:00:00,Veneto,Vicenza,141.0


In [15]:
#Translate "In fase" item to English 
data_english['PROVINCE NAME'] = data_english['PROVINCE NAME'].replace(
    {'In fase di definizione/aggiornamento': 'N/A'})
data_english

,DATE,REGION NAME,PROVINCE NAME,TOTAL CASES
0,2020-02-24 18:00:00,Abruzzo,Chieti,0.0
1,2020-02-24 18:00:00,Abruzzo,L'Aquila,0.0
2,2020-02-24 18:00:00,Abruzzo,Pescara,0.0
3,2020-02-24 18:00:00,Abruzzo,Teramo,0.0
4,2020-02-24 18:00:00,Abruzzo,N/A,0.0
...,...,...,...,...
2427,2020-03-13 17:00:00,Veneto,Treviso,327.0
2428,2020-03-13 17:00:00,Veneto,Venezia,248.0
2429,2020-03-13 17:00:00,Veneto,Verona,210.0
2430,2020-03-13 17:00:00,Veneto,Vicenza,141.0


In [16]:
# Convert to SQL 
final_data = data_english

In [17]:
# Connect to SQL database
rds_connection_string = "osedjan:@localhost:5432/ETL Project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [18]:
# Create Engine 
engine.table_names()

['Corona1']

In [19]:
# Send to SQL 
final_data.to_sql(name='Corona1', con=engine, if_exists='append', index=False)


In [20]:
# Read from SQL 
pd.read_sql_query('select * from Corona1', con=engine).head()

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "corona1" does not exist
LINE 1: select * from Corona1
                      ^

[SQL: select * from Corona1]
(Background on this error at: http://sqlalche.me/e/f405)